In [92]:
#IMPORTS
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [93]:
def load_data(df):
    y=df.quality
    X=df.drop('quality',1)
    X = min_max_scaler.fit_transform(X)
    return X, y

In [94]:
labelencoder = LabelEncoder()
min_max_scaler = preprocessing.MinMaxScaler()

#CREATE AND MERGE DATAFRAMES
dfRed=pd.read_csv("winequality-red.csv", sep=";")
dfWhite=pd.read_csv("winequality-white.csv", sep=";")

dfRed['Type']="Red"
dfWhite['Type']="White"
df=pd.concat([dfRed,dfWhite])

In [95]:
df.dropna(inplace=True)

In [96]:
df['Type']=labelencoder.fit_transform(df['Type'])
df = df.sample(frac=1)

X, y = load_data(df)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

C:\Users\walli\AppData\Local\Temp/ipykernel_13392/3104105429.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X=df.drop('quality',1)


In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6497 entries, 2587 to 2255
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         6497 non-null   float64
 1   volatile acidity      6497 non-null   float64
 2   citric acid           6497 non-null   float64
 3   residual sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free sulfur dioxide   6497 non-null   float64
 6   total sulfur dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
 12  Type                  6497 non-null   int32  
dtypes: float64(11), int32(1), int64(1)
memory usage: 685.2 KB


In [98]:
#LOGISTIC REGRESSION

logreg = LogisticRegression(max_iter=5000)
logreg.fit(X_train,y_train)

y_predict = logreg.predict(X_test)

# dfLogReg=pd.DataFrame()
# dfLogReg['Real'] = y_test
# dfLogReg['Prediction']=y_predict
# print(dfLogReg.head())

print(classification_report(y_test, y_predict))

training_accuracy = logreg.score(X_train,y_train)
print('training_accuracy:',training_accuracy)

testing_accuracy = logreg.score(X_test,y_test)
print('testing_accuracy:',testing_accuracy)

#TODO: Perquè dona aquells warnings?

              precision    recall  f1-score   support

           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00        41
           5       0.61      0.64      0.62       423
           6       0.57      0.74      0.64       593
           7       0.49      0.20      0.29       203
           8       0.00      0.00      0.00        37

    accuracy                           0.58      1300
   macro avg       0.28      0.26      0.26      1300
weighted avg       0.53      0.58      0.54      1300

training_accuracy: 0.536078506830864
testing_accuracy: 0.5769230769230769


C:\Users\walli\miniconda3\envs\ml-environment\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\walli\miniconda3\envs\ml-environment\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\walli\miniconda3\envs\ml-environment\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

In [99]:
#PERCEPTRON
perceptron = Perceptron()
perceptron.fit(X_train, y_train)

y_predict=perceptron.predict(X_test)

# dfPerceptron=pd.DataFrame()
# dfPerceptron['Real'] = y_test
# dfPerceptron['Prediction']=y_predict
# print(dfPerceptron.head)

print(classification_report(y_test, y_predict))

training_accuracy = perceptron.score(X_train,y_train)
print('training_accuracy:',training_accuracy)

testing_accuracy = perceptron.score(X_test,y_test)
print('testing_accuracy:',testing_accuracy)

              precision    recall  f1-score   support

           3       0.00      0.00      0.00         3
           4       0.19      0.12      0.15        41
           5       0.51      0.37      0.43       423
           6       0.48      0.49      0.48       593
           7       0.57      0.08      0.14       203
           8       0.09      0.76      0.15        37

    accuracy                           0.38      1300
   macro avg       0.30      0.30      0.22      1300
weighted avg       0.48      0.38      0.39      1300

training_accuracy: 0.3500096209351549
testing_accuracy: 0.38


C:\Users\walli\miniconda3\envs\ml-environment\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\walli\miniconda3\envs\ml-environment\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\walli\miniconda3\envs\ml-environment\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

In [100]:
#RANDOM FOREST
rf = RandomForestClassifier(max_depth=14)
rf.fit(X_train, y_train)

y_predict=rf.predict(X_test)

# dfRF=pd.DataFrame()
# dfRF['Real'] = y_test
# dfRF['Prediction']=y_predict
# print(dfRF.head)

print(classification_report(y_test, y_predict))

training_accuracy = rf.score(X_train,y_train)
print('training_accuracy:',training_accuracy)

testing_accuracy = rf.score(X_test,y_test)
print('testing_accuracy:',testing_accuracy)

              precision    recall  f1-score   support

           3       0.00      0.00      0.00         3
           4       1.00      0.07      0.14        41
           5       0.71      0.72      0.72       423
           6       0.68      0.81      0.74       593
           7       0.73      0.53      0.61       203
           8       1.00      0.38      0.55        37

    accuracy                           0.70      1300
   macro avg       0.69      0.42      0.46      1300
weighted avg       0.71      0.70      0.68      1300

training_accuracy: 0.9780642678468348
testing_accuracy: 0.6984615384615385


C:\Users\walli\miniconda3\envs\ml-environment\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\walli\miniconda3\envs\ml-environment\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\walli\miniconda3\envs\ml-environment\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r